# 모델링

In [ ]:
#한글 깨짐 -> 설치 후 런타임 다시 시작 필요!
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 0s (48.6 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

In [ ]:
# MeCab, nltk 설치
!pip install nltk
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-09-01 11:43:08--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2022-09-01 11:43:08--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce6dcff0ba2a6acb8632ef293fb.dl.dropboxusercontent.com/cd/0/get/BsF4kUni5bD0-Lby02xSPym1EFCKgLsuElAMGueN5hHSvLVbCL0pmX1IRU8oMxvkNi4lSLsKPg8bKJRacnF5sf6vlZXtW2QWBv9C6v_fMD-7q7UYMGhM7aiDUdR8Bzz_Ak84sbioaUbjjGAXZHXxl9Xj-jtu-LNcjUX03QN7X-pChNKTolhrEJ1aiTCBLRbhMMs/file?dl=1# [following]
--2022-

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import random as rn
import re
import os
import warnings
import folium
import geopy.distance
import konlpy
import MeCab
from collections import Counter
from konlpy.tag import Kkma, Komoran, Hannanum, Okt, Mecab
from konlpy.utils import pprint
from nltk import Text
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from keras.layers import Dense, Activation
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold


plt.rc('font',  family='NanumBarunGothic')
plt.rcParams["axes.unicode_minus"]=False #마이너스 깨짐 해결
%matplotlib inline
warnings.filterwarnings(action='ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## (1) 데이터 불러오기

In [ ]:
data="/content/drive/MyDrive/Project/팀 프로젝트/3_company_dinner/data/"

In [ ]:
df = pd.read_csv(os.path.join(data,"company_dinner_tokenization2019.csv"),index_col=0)
df=df.reset_index(drop=True)
df['review']=df['review'].fillna("")
df['review_check']=df['review_check'].fillna("")
df.head()

,store_name,date,review,length,review_check,check_length,menu,mood,service,price,...,삼거리,맛,맛집,메뉴_음식,친절_직원,때,예약,가격,분위기,추천
0,백억하누 강남본점,2020-01-03,네이버로 예약하고 방문했는데 사정이 있어 조용한 구석자리를 부탁한다고 썼는데도 구석...,358,네이버로 예약하고 방문했는데 사정이 있어 조용한 구석 자리를 부탁한다고 썼는데도 구...,370,1,1,1,1,...,0,0,0,1,1,0,1,0,0,0
1,강남진해장,2022-04-05,일반적인 양평해장국과 다르게 기본 간이 되어있고 베이스가 육개장과 뼈해장국같은 느...,343,일반적인 양평 해장국과 다르게 기본 간이 되어 있고 베이스가 육개장과 뼈해장국 같은...,348,1,1,1,1,...,0,1,0,1,0,0,0,1,0,0
2,백억하누 강남본점,2022-05-07,개인룸이 있고 룸안에 화장실이 있어 너무 편하고 좋았는데 옆에 개인 서랍이 있는지...,163,개인 룸이 있고 룸 안에 화장실이 있어 너무 편하고 좋았는데 옆에 개인 서랍이 있는...,165,1,1,1,0,...,0,1,0,1,1,0,0,0,0,0
3,딘타이펑 강남점,2019-11-21,분위기도 괜찮고 맛있어요 네이버예약 덕분에 에이드도 서비스로 나왔네요 가격은 비싸지...,59,분위기도 괜찮고 맛있어요 네이버예약 덕분에 에이드도 서비스로 나왔네요 가격은 비싸지...,59,1,1,1,1,...,0,1,0,0,0,0,1,1,1,0
4,백억하누 강남본점,2021-05-25,룸으로 이용했는데 화장실도 딸려있어서 너무 편했어요 삼합 재료들이 신선해서 생각보다...,265,룸으로 이용했는데 화장실도 딸려 있어서 너무 편했어요 삼합 재료들이 신선해서 생각보...,276,1,1,0,1,...,0,1,0,1,0,0,0,0,0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7822 entries, 0 to 7821
Data columns (total 48 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   store_name    7822 non-null   object
 1   date          7822 non-null   object
 2   review        7822 non-null   object
 3   length        7822 non-null   int64 
 4   review_check  7822 non-null   object
 5   check_length  7822 non-null   int64 
 6   menu          7822 non-null   int64 
 7   mood          7822 non-null   int64 
 8   service       7822 non-null   int64 
 9   price         7822 non-null   int64 
 10  clean         7822 non-null   int64 
 11  room          7822 non-null   int64 
 12  parking       7822 non-null   int64 
 13  photo         7822 non-null   int64 
 14  sum           7822 non-null   int64 
 15  helpful       7822 non-null   int64 
 16  store         7822 non-null   int64 
 17  딘타이펑          7822 non-null   int64 
 18  백억하누          7822 non-null   int64 
 19  느린마을  

In [ ]:
df.columns

Index(['store_name', 'date', 'review', 'length', 'review_check',
       'check_length', 'menu', 'mood', 'service', 'price', 'clean', 'room',
       'parking', 'photo', 'sum', 'helpful', 'store', '딘타이펑', '백억하누', '느린마을',
       '화기애애', '라그릴리아', '강남진해장', '구스', '파파야리프', '고양이부엌', '시골야채된장', '을밀대', '담소',
       '청류벽', '기리야마', '시카고', '대려도', '신부자', '봉추찜닭', '리샨', '봉피양', 'Pavox', '삼거리',
       '맛', '맛집', '메뉴_음식', '친절_직원', '때', '예약', '가격', '분위기', '추천'],
      dtype='object')

In [ ]:
df['store_name'].unique(), len(df['store_name'].unique())

(array(['백억하누 강남본점', '강남진해장', '딘타이펑 강남점', '삼거리포차', '화기애애 강남역점',
        '느린마을양조장 강남점', '시골야채된장', '봉피양제주 강남점', '라그릴리아 SPC스퀘어점', '청류벽',
        '대려도', '신부자식당', '리샨 삼성타운점', '구스아일랜드 브루하우스', '을밀대 강남점', '파파야리프',
        '고양이부엌', '오리지널시카고피자', 'Pavox', '우동명가기리야마본진', '담소사골순대 강남본점',
        '봉추찜닭 강남주공공이점'], dtype=object), 22)

## (2) 토큰화 함수 만들기

### 1) getNVM_lemma
- 모든 태그 사용

In [ ]:
def getNVM_lemma(text):
    tokenizer = MeCab.Tagger()
    parsed = tokenizer.parse(text)
    word_tag = [w for w in parsed.split("\n")] 
    pos = []

    for word_ in word_tag[:-2]:
        word = word_.split('\t') #['아버지', 'NNG,*,F,아버지,*,*,*,*']
        pos.append(word[0])
    return pos

### 2) getNVM_lemma2 + 불용어
- 하기 태그만 사용 + 불용어 제거
- 'NNG','NNP', 'SN', 'NR', 'VA', 'VV', 'VCN', 'VCP', 'XPN', 'XR', 'XSN', 'NNBC'만 이용

In [ ]:
stop_words = ['한우','하','포장','평양냉면','탕','타이','전복','전','재','우동','오','안주','소고기','세트','샤오','새우','삼합','사람','볶음밥','보','바오','면','맥주','말','막국수','막걸리','룽','떡볶이','딤섬','딘','들기름','들','되','느끼','네이버','냉면','나오','김치','굽','국물','고기','강남','가']
len(stop_words)
def getNVM_lemma2(text):
    tokenizer = MeCab.Tagger()
    parsed = tokenizer.parse(text)
    #print(parsed)
    word_tag = [w for w in parsed.split("\n")] 
    pos = []
    tags = ['NNG','NNP', 'SN', 'NR', 'VA', 'VV', 'VCN', 'VCP', 'XPN', 'XR', 'NNBC']
   
    for word_ in word_tag[:-2]:
        word = word_.split('\t') #['아버지', 'NNG,*,F,아버지,*,*,*,*']
        tag = word[1].split(",")  #['EC', '*', 'F', '는다', '*', '*', '*', '*']
        if('+' in tag[0]): #단어가 여러 형태소로 구성된 경우
            if ('VV' in tag[0] or 'VA' in tag[0]):
                t = tag[-1].split('/')[0]
                if t not in stop_words:
                    pos.append(t)
        if ((tag[0] in tags) and (word[0] not in stop_words)):
          pos.append(word[0])
    return pos

## (3) Logistic Regression

### 1) tag 안 쓴 버전

#### x (feature)

In [ ]:
tdm_tfidf = tfidf.fit_transform(df['review_check'].values.astype('U'))

In [ ]:
# x_tf : 리뷰(모든 tag 사용), 리뷰길이, 가게명
x_tf = tdm_tfidf.toarray()
x_tf_len = []
for i in range(len(x_tf)):
    a = x_tf[i]
    a = np.append(a, np.log(df['check_length'][i]+1)) # 길이 컬럼 추가

    a = np.append(a, df['딘타이펑'][i])    
    a = np.append(a, df['백억하누'][i])
    a = np.append(a, df['느린마을'][i])   
    a = np.append(a, df['화기애애'][i])    
    a = np.append(a, df['라그릴리아'][i])    
    a = np.append(a, df['강남진해장'][i])    
    a = np.append(a, df['구스'][i])
    a = np.append(a, df['파파야리프'][i])   
    a = np.append(a, df['고양이부엌'][i])    
    a = np.append(a, df['시골야채된장'][i])    
    a = np.append(a, df['을밀대'][i])    
    a = np.append(a, df['담소'][i])
    a = np.append(a, df['청류벽'][i])    
    a = np.append(a, df['기리야마'][i])
    a = np.append(a, df['신부자'][i])    
    a = np.append(a, df['시카고'][i])    
    a = np.append(a, df['대려도'][i])    
    a = np.append(a, df['봉추찜닭'][i])    
    a = np.append(a, df['리샨'][i]) 

    x_tf_len.append(a)

In [ ]:
# x_cv : 리뷰(모든 tag 사용), 리뷰길이, 가게명, 키워드 전체
x_cv = tdm_cv.toarray() # TDM (countvector) 추가
x_cv_len = []
for i in range(len(x_cv)):
    a = x_cv[i]
    a = np.append(a,np.log(df['check_length'][i]+1)) # 길이 컬럼 추가
 
    a = np.append(a, df['딘타이펑'][i])    
    a = np.append(a, df['백억하누'][i])
    a = np.append(a, df['느린마을'][i])   
    a = np.append(a, df['화기애애'][i])    
    a = np.append(a, df['라그릴리아'][i])    
    a = np.append(a, df['강남진해장'][i])    
    a = np.append(a, df['구스'][i])
    a = np.append(a, df['파파야리프'][i])   
    a = np.append(a, df['고양이부엌'][i])    
    a = np.append(a, df['시골야채된장'][i])    
    a = np.append(a, df['을밀대'][i])    
    a = np.append(a, df['담소'][i])
    a = np.append(a, df['청류벽'][i])    
    a = np.append(a, df['기리야마'][i])
    a = np.append(a, df['신부자'][i])    
    a = np.append(a, df['시카고'][i])    
    a = np.append(a, df['대려도'][i])    
    a = np.append(a, df['봉추찜닭'][i])    
    a = np.append(a, df['리샨'][i])
    a = np.append(a, df['봉피양'][i])       
    a = np.append(a, df['Pavox'][i])    
    a = np.append(a, df['삼거리'][i])  
 
    a = np.append(a, df['맛'][i])
    a = np.append(a, df['맛집'][i])
    a = np.append(a, df['메뉴_음식'][i])
    a = np.append(a, df['친절_직원'][i])
    a = np.append(a, df['때'][i])
    a = np.append(a, df['예약'][i])
    a = np.append(a, df['가격'][i])
    a = np.append(a, df['분위기'][i])
    a = np.append(a, df['추천'][i])  

    x_cv_len.append(a)

#### y(target)

In [ ]:
y = df['helpful']

#### 학습하기

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_cv_len, y, test_size=0.2, random_state=42)

In [ ]:
clf_cv = LogisticRegression(random_state = 42, C=0.1, max_iter = 100).fit(x_train,y_train)
pred = clf_cv.predict(x_test)
accuracy_score(y_test, pred)

0.8856230031948882

In [ ]:
x2_train, x2_test, y2_train, y2_test = train_test_split(x_tf_len, y, test_size=0.2, random_state=42)

In [ ]:
clf_tf = LogisticRegression(random_state = 42, C=0.1, max_iter = 100).fit(x2_train,y2_train)
pred2 = clf_tf.predict(x2_test)
accuracy_score(y2_test, pred2)

0.8619808306709266

### 2) 모든 tag 다 쓴 버전

#### tfidf, count Vector

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,2),tokenizer = getNVM_lemma, preprocessor = None, lowercase = False)
cv = CountVectorizer(ngram_range=(1,2),tokenizer = getNVM_lemma, preprocessor = None, lowercase = False)
tdm_tfidf = tfidf.fit_transform(df['review_check'].values.astype('U'))
tdm_cv= cv.fit_transform(df['review_check'].values.astype('U'))

#### x (feature)

In [ ]:
# x_tf : 리뷰(모든 tag 사용), 리뷰길이, 가게명
x_tf = tdm_tfidf.toarray()
x_tf_len = []
for i in range(len(x_tf)):
    a = x_tf[i]
    a = np.append(a, np.log(df['check_length'][i]+1)) # 길이 컬럼 추가

    a = np.append(a, df['딘타이펑'][i])    
    a = np.append(a, df['백억하누'][i])
    a = np.append(a, df['느린마을'][i])   
    a = np.append(a, df['화기애애'][i])    
    a = np.append(a, df['라그릴리아'][i])    
    a = np.append(a, df['강남진해장'][i])    
    a = np.append(a, df['구스'][i])
    a = np.append(a, df['파파야리프'][i])   
    a = np.append(a, df['고양이부엌'][i])    
    a = np.append(a, df['시골야채된장'][i])    
    a = np.append(a, df['을밀대'][i])    
    a = np.append(a, df['담소'][i])
    a = np.append(a, df['청류벽'][i])    
    a = np.append(a, df['기리야마'][i])
    a = np.append(a, df['신부자'][i])    
    a = np.append(a, df['시카고'][i])    
    a = np.append(a, df['대려도'][i])    
    a = np.append(a, df['봉추찜닭'][i])    
    a = np.append(a, df['리샨'][i]) 

    x_tf_len.append(a)

In [ ]:
# x_cv : 리뷰(모든 tag 사용), 리뷰길이, 가게명, 키워드 전체
x_cv = tdm_cv.toarray() # TDM (countvector) 추가
x_cv_len = []
for i in range(len(x_cv)):
    a = x_cv[i]
    a = np.append(a,np.log(df['check_length'][i]+1)) # 길이 컬럼 추가
 
    a = np.append(a, df['딘타이펑'][i])    
    a = np.append(a, df['백억하누'][i])
    a = np.append(a, df['느린마을'][i])   
    a = np.append(a, df['화기애애'][i])    
    a = np.append(a, df['라그릴리아'][i])    
    a = np.append(a, df['강남진해장'][i])    
    a = np.append(a, df['구스'][i])
    a = np.append(a, df['파파야리프'][i])   
    a = np.append(a, df['고양이부엌'][i])    
    a = np.append(a, df['시골야채된장'][i])    
    a = np.append(a, df['을밀대'][i])    
    a = np.append(a, df['담소'][i])
    a = np.append(a, df['청류벽'][i])    
    a = np.append(a, df['기리야마'][i])
    a = np.append(a, df['신부자'][i])    
    a = np.append(a, df['시카고'][i])    
    a = np.append(a, df['대려도'][i])    
    a = np.append(a, df['봉추찜닭'][i])    
    a = np.append(a, df['리샨'][i])
    a = np.append(a, df['봉피양'][i])       
    a = np.append(a, df['Pavox'][i])    
    a = np.append(a, df['삼거리'][i])  
 
    a = np.append(a, df['맛'][i])
    a = np.append(a, df['맛집'][i])
    a = np.append(a, df['메뉴_음식'][i])
    a = np.append(a, df['친절_직원'][i])
    a = np.append(a, df['때'][i])
    a = np.append(a, df['예약'][i])
    a = np.append(a, df['가격'][i])
    a = np.append(a, df['분위기'][i])
    a = np.append(a, df['추천'][i])  

    x_cv_len.append(a)

#### y(target)

In [ ]:
y = df['helpful']

#### 학습하기

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_cv_len, y, test_size=0.2, random_state=42)

In [ ]:
clf_cv = LogisticRegression(random_state = 42, C=0.1, max_iter = 100).fit(x_train,y_train)
pred = clf_cv.predict(x_test)
accuracy_score(y_test, pred)

0.8869009584664537

In [ ]:
x2_train, x2_test, y2_train, y2_test = train_test_split(x_tf_len, y, test_size=0.2, random_state=42)

In [ ]:
clf_tf = LogisticRegression(random_state = 42, C=0.1, max_iter = 100).fit(x2_train,y2_train)
pred2 = clf_tf.predict(x2_test)
accuracy_score(y2_test, pred2)

0.8613418530351438

### 3)CV 기준으로 바꾼 버전

#### 2) x (feature)

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,2),tokenizer = getNVM_lemma2, preprocessor = None, lowercase = False)
cv = CountVectorizer(ngram_range=(1,2),tokenizer = getNVM_lemma2, preprocessor = None, lowercase = False)
tdm_tfidf = tfidf.fit_transform(df['review_check'].values.astype('U'))
tdm_cv= cv.fit_transform(df['review_check'].values.astype('U'))

In [ ]:
# x_cv_1 : 리뷰, 리뷰길이, 가게명, 키워드 전체
x_cv_1 = tdm_cv.toarray() # TDM (countvector) 추가
x_cv_len_1 = []
for i in range(len(x_cv_1)):
    a = x_cv_1[i]
    a = np.append(a,np.log(df['check_length'][i]+1)) # 길이 컬럼 추가
 
    a = np.append(a, df['딘타이펑'][i])    
    a = np.append(a, df['백억하누'][i])
    a = np.append(a, df['느린마을'][i])   
    a = np.append(a, df['화기애애'][i])    
    a = np.append(a, df['라그릴리아'][i])    
    a = np.append(a, df['강남진해장'][i])    
    a = np.append(a, df['구스'][i])
    a = np.append(a, df['파파야리프'][i])   
    a = np.append(a, df['고양이부엌'][i])    
    a = np.append(a, df['시골야채된장'][i])    
    a = np.append(a, df['을밀대'][i])    
    a = np.append(a, df['담소'][i])
    a = np.append(a, df['청류벽'][i])    
    a = np.append(a, df['기리야마'][i])
    a = np.append(a, df['신부자'][i])    
    a = np.append(a, df['시카고'][i])    
    a = np.append(a, df['대려도'][i])    
    a = np.append(a, df['봉추찜닭'][i])    
    a = np.append(a, df['리샨'][i])
    a = np.append(a, df['봉피양'][i])       
    a = np.append(a, df['Pavox'][i])    
    a = np.append(a, df['삼거리'][i])  
 
    a = np.append(a, df['맛'][i])
    a = np.append(a, df['맛집'][i])
    a = np.append(a, df['메뉴_음식'][i])
    a = np.append(a, df['친절_직원'][i])
    a = np.append(a, df['때'][i])
    a = np.append(a, df['예약'][i])
    a = np.append(a, df['가격'][i])
    a = np.append(a, df['분위기'][i])
    a = np.append(a, df['추천'][i])  

    x_cv_len_1.append(a)

In [ ]:
# x_cv_2 : 리뷰, 리뷰길이, 가게명, 키워드(맛, 서비스(친절_직원) 제외)
x_cv_2 = tdm_cv.toarray() # TDM (countvector) 추가
x_cv_len_2 = []
for i in range(len(x_cv_2)):
    a = x_cv_2[i]
    a = np.append(a,np.log(df['check_length'][i]+1)) # 길이 컬럼 추가
 
    a = np.append(a, df['딘타이펑'][i])    
    a = np.append(a, df['백억하누'][i])
    a = np.append(a, df['느린마을'][i])   
    a = np.append(a, df['화기애애'][i])    
    a = np.append(a, df['라그릴리아'][i])    
    a = np.append(a, df['강남진해장'][i])    
    a = np.append(a, df['구스'][i])
    a = np.append(a, df['파파야리프'][i])   
    a = np.append(a, df['고양이부엌'][i])    
    a = np.append(a, df['시골야채된장'][i])    
    a = np.append(a, df['을밀대'][i])    
    a = np.append(a, df['담소'][i])
    a = np.append(a, df['청류벽'][i])    
    a = np.append(a, df['기리야마'][i])
    a = np.append(a, df['신부자'][i])    
    a = np.append(a, df['시카고'][i])    
    a = np.append(a, df['대려도'][i])    
    a = np.append(a, df['봉추찜닭'][i])    
    a = np.append(a, df['리샨'][i])
    a = np.append(a, df['봉피양'][i])       
    a = np.append(a, df['Pavox'][i])    
    a = np.append(a, df['삼거리'][i])  
 
    a = np.append(a, df['맛집'][i])
    a = np.append(a, df['메뉴_음식'][i])
    a = np.append(a, df['때'][i])
    a = np.append(a, df['예약'][i])
    a = np.append(a, df['가격'][i])
    a = np.append(a, df['분위기'][i])
    a = np.append(a, df['추천'][i])  

    x_cv_len_2.append(a)

In [ ]:
# x_cv_3 : 리뷰, 리뷰길이, 가게명, 키워드(맛, 맛집, 메뉴_음식 제외)
x_cv_3 = tdm_cv.toarray() # TDM (countvector) 추가
x_cv_len_3 = []
for i in range(len(x_cv_3)):
    a = x_cv_3[i]
    a = np.append(a,np.log(df['check_length'][i]+1)) # 길이 컬럼 추가
 
    a = np.append(a, df['딘타이펑'][i])    
    a = np.append(a, df['백억하누'][i])
    a = np.append(a, df['느린마을'][i])   
    a = np.append(a, df['화기애애'][i])    
    a = np.append(a, df['라그릴리아'][i])    
    a = np.append(a, df['강남진해장'][i])    
    a = np.append(a, df['구스'][i])
    a = np.append(a, df['파파야리프'][i])   
    a = np.append(a, df['고양이부엌'][i])    
    a = np.append(a, df['시골야채된장'][i])    
    a = np.append(a, df['을밀대'][i])    
    a = np.append(a, df['담소'][i])
    a = np.append(a, df['청류벽'][i])    
    a = np.append(a, df['기리야마'][i])
    a = np.append(a, df['신부자'][i])    
    a = np.append(a, df['시카고'][i])    
    a = np.append(a, df['대려도'][i])    
    a = np.append(a, df['봉추찜닭'][i])    
    a = np.append(a, df['리샨'][i])
    a = np.append(a, df['봉피양'][i])       
    a = np.append(a, df['Pavox'][i])    
    a = np.append(a, df['삼거리'][i])  
 
    a = np.append(a, df['친절_직원'][i])
    a = np.append(a, df['때'][i])
    a = np.append(a, df['예약'][i])
    a = np.append(a, df['가격'][i])
    a = np.append(a, df['분위기'][i])
    a = np.append(a, df['추천'][i]) 

    x_cv_len_3.append(a)

In [ ]:
# x_cv_4 : 리뷰, 리뷰길이, 가게명, 키워드(맛, 맛집, 분위기 제외)
x_cv_4 = tdm_cv.toarray() # TDM (countvector) 추가
x_cv_len_4 = []
for i in range(len(x_cv_4)):
    a = x_cv_4[i]
    a = np.append(a,np.log(df['check_length'][i]+1)) # 길이 컬럼 추가
 
    a = np.append(a, df['딘타이펑'][i])    
    a = np.append(a, df['백억하누'][i])
    a = np.append(a, df['느린마을'][i])   
    a = np.append(a, df['화기애애'][i])    
    a = np.append(a, df['라그릴리아'][i])    
    a = np.append(a, df['강남진해장'][i])    
    a = np.append(a, df['구스'][i])
    a = np.append(a, df['파파야리프'][i])   
    a = np.append(a, df['고양이부엌'][i])    
    a = np.append(a, df['시골야채된장'][i])    
    a = np.append(a, df['을밀대'][i])    
    a = np.append(a, df['담소'][i])
    a = np.append(a, df['청류벽'][i])    
    a = np.append(a, df['기리야마'][i])
    a = np.append(a, df['신부자'][i])    
    a = np.append(a, df['시카고'][i])    
    a = np.append(a, df['대려도'][i])    
    a = np.append(a, df['봉추찜닭'][i])    
    a = np.append(a, df['리샨'][i])
    a = np.append(a, df['봉피양'][i])       
    a = np.append(a, df['Pavox'][i])    
    a = np.append(a, df['삼거리'][i])  
 
    a = np.append(a, df['메뉴_음식'][i])
    a = np.append(a, df['친절_직원'][i])
    a = np.append(a, df['때'][i])
    a = np.append(a, df['예약'][i])
    a = np.append(a, df['가격'][i])
    a = np.append(a, df['추천'][i])  

    x_cv_len_4.append(a)

In [ ]:
# x_cv_5 : 리뷰, 키워드 전체
x_cv_5 = tdm_cv.toarray() # TDM (countvector) 추가
x_cv_len_5 = []
for i in range(len(x_cv_5)):
    a = x_cv_5[i]

    a = np.append(a, df['맛'][i])
    a = np.append(a, df['맛집'][i])
    a = np.append(a, df['메뉴_음식'][i])
    a = np.append(a, df['친절_직원'][i])
    a = np.append(a, df['때'][i])
    a = np.append(a, df['예약'][i])
    a = np.append(a, df['가격'][i])
    a = np.append(a, df['분위기'][i])
    a = np.append(a, df['추천'][i])  

    x_cv_len_5.append(a)

In [ ]:
# x_cv_6 : 리뷰, 가게명, 키워드 전체
x_cv_6 = tdm_cv.toarray() # TDM (countvector) 추가
x_cv_len_6 = []
for i in range(len(x_cv_6)):
    a = x_cv_6[i]

    a = np.append(a, df['딘타이펑'][i])    
    a = np.append(a, df['백억하누'][i])
    a = np.append(a, df['느린마을'][i])   
    a = np.append(a, df['화기애애'][i])    
    a = np.append(a, df['라그릴리아'][i])    
    a = np.append(a, df['강남진해장'][i])    
    a = np.append(a, df['구스'][i])
    a = np.append(a, df['파파야리프'][i])   
    a = np.append(a, df['고양이부엌'][i])    
    a = np.append(a, df['시골야채된장'][i])    
    a = np.append(a, df['을밀대'][i])    
    a = np.append(a, df['담소'][i])
    a = np.append(a, df['청류벽'][i])    
    a = np.append(a, df['기리야마'][i])
    a = np.append(a, df['신부자'][i])    
    a = np.append(a, df['시카고'][i])    
    a = np.append(a, df['대려도'][i])    
    a = np.append(a, df['봉추찜닭'][i])    
    a = np.append(a, df['리샨'][i])
    a = np.append(a, df['봉피양'][i])       
    a = np.append(a, df['Pavox'][i])    
    a = np.append(a, df['삼거리'][i])  
 
    a = np.append(a, df['맛'][i])
    a = np.append(a, df['맛집'][i])
    a = np.append(a, df['메뉴_음식'][i])
    a = np.append(a, df['친절_직원'][i])
    a = np.append(a, df['때'][i])
    a = np.append(a, df['예약'][i])
    a = np.append(a, df['가격'][i])
    a = np.append(a, df['분위기'][i])
    a = np.append(a, df['추천'][i])  

    x_cv_len_6.append(a)

In [ ]:
len(x_cv_len_1[0]), len(x_cv_len_2[0]), len(x_cv_len_3[0]), len(x_cv_len_4[0]), len(x_cv_len_5[0]), len(x_cv_len_6[0])

(28777, 28775, 28774, 28774, 28754, 28776)

#### 3) y(target)

In [ ]:
y = df['helpful']

#### 4) 학습하기

##### x_cv_len_1

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_cv_len_1, y, test_size=0.2, random_state=42)

In [ ]:
clf_cv_1 = LogisticRegression(random_state = 42, C=0.1, max_iter = 100).fit(x_train,y_train)
pred = clf_cv_1.predict(x_test)
accuracy_score(y_test, pred)

0.8830670926517572

x_cv_len_2

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_cv_len_2, y, test_size=0.2, random_state=42)

In [ ]:
clf_cv_2 = LogisticRegression(random_state = 42, C=0.1, max_iter = 100).fit(x_train,y_train)
pred = clf_cv_2.predict(x_test)
accuracy_score(y_test, pred)

0.8830670926517572

x_cv_len_3

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_cv_len_3, y, test_size=0.2, random_state=42)

In [ ]:
clf_cv_3 = LogisticRegression(random_state = 42, C=0.1, max_iter = 100).fit(x_train,y_train)
pred = clf_cv_3.predict(x_test)
accuracy_score(y_test, pred)

0.8837060702875399

x_cv_len_4

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_cv_len_4, y, test_size=0.2, random_state=42)

In [ ]:
clf_cv_4 = LogisticRegression(random_state = 42, C=0.1, max_iter = 100).fit(x_train,y_train)
pred = clf_cv_4.predict(x_test)
accuracy_score(y_test, pred)

0.8830670926517572

x_cv_len_5

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_cv_len_5, y, test_size=0.2, random_state=42)

In [ ]:
clf_cv_5 = LogisticRegression(random_state = 42, C=0.1, max_iter = 100).fit(x_train,y_train)
pred = clf_cv_5.predict(x_test)
accuracy_score(y_test, pred)

0.8837060702875399

x_cv_len_6

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_cv_len_6, y, test_size=0.2, random_state=42)

In [ ]:
clf_cv_6 = LogisticRegression(random_state = 42, C=0.1, max_iter = 100).fit(x_train,y_train)
pred = clf_cv_6.predict(x_test)
accuracy_score(y_test, pred)

0.8856230031948882

##### 결과
- x_cv_len_1: 88.31%
- x_cv_len_2: 88.31%
- x_cv_len_3: 88.37%
- x_cv_len_4: 88.31%
- x_cv_len_5: 88.37%
- x_cv_len_6: 88.56%

### 가게 정리
---------------------
- df: 모든 가게 있는 그룹
- df2: 리뷰 개수가 제일 많은(1000개 이상되는) 식당 제거 그룹
	- 제거된 식당: 딘타이펑, 백억하누
- df3: 리뷰 개수가 많은 상위 7개 식당 제거 그룹
  - 제거된 식당: 딘타이펑, 백억하누, 느린마을양조장, 화기애애, 라그릴리아, 강남진해장, 구스아일랜드
- df4: 리뷰 개수가 중간층에 있는 식당 8개 제거 그룹
  - 제거된 식당: 파파야리프, 고양리부엌, 시골야채된장, 을밀대, 담소사골순대, 청류벽, 우동명가기리야마본진, 신부자식당

cf vs tf
- cf : review, review 길이, 가게 항목들, 키워드 항목들
- tf : review, review 길이, 가게 항목들

#### lemma1 정리 (cf vs tf)
tag: NNP, NNG
- df: 88% vs 85.9%
- df2: 89.6% vs 86.3%
- df3: 90.6% vs 90.7%
- df4: 85.6% vs 81.9%

#### lemma1 정리 (cf vs tf)
tag: NNP, NNG + 불용어 제거
- df: 87.8% vs 85.8%
- df2: 89.2% vs 86.4%
- df3: 91.1% vs 90.7%
- df4: 85.6% vs 82.0%

#### lemma2 정리 (cf vs tf)
tag : 'NNG','NNP', 'SN', 'NR', 'VA', 'VV', 'VCN', 'VCP', 'XPN', 'XR', 'XSN', 'NNBC'
- df: 88.50% vs 86.13%
- df2: 89.20% vs 86.24%
- df3: 91.07% vs 90.74%
- df4: 85.93% vs 81.76%

#### lemma2+stopword 정리 (cf vs tf)
tag : 'NNG','NNP', 'SN', 'NR', 'VA', 'VV', 'VCN', 'VCP', 'XPN', 'XR', 'XSN', 'NNBC' + 불용어 제거
- df: 88.31% vs 85.88%
- df2: 89.84% vs 86.33%
- df3: 91.24% vs 90.74%
- df4: 85.21% vs 81.76%
- df5: 88.83% vs 85.52%

#### lemma2+stopword+cv
- x_cv_1 : 리뷰, 리뷰길이, 가게명, 키워드 전체
- x_cv_2 : 리뷰, 리뷰길이, 가게명, 키워드(맛, 서비스(친절_직원) 제외)
- x_cv_3 : 리뷰, 리뷰길이, 가게명, 키워드(맛, 맛집, 메뉴_음식 제외)
- x_cv_4 : 리뷰, 리뷰길이, 가게명, 키워드(맛, 맛집, 분위기 제외)
- x_cv_5 : 리뷰, 키워드 전체
- x_cv_6 : 리뷰, 가게명, 키워드 전체
-----------
##### 결과
- x_cv_len_1: 88.31%
- x_cv_len_2: 88.31%
- x_cv_len_3: 88.37%
- x_cv_len_4: 88.31%
- x_cv_len_5: 88.37%
- x_cv_len_6: 88.56%